In [24]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
from train import train, train_and_evaluate

import utils
import model.net as net
import model.data_loader as data_loader
from evaluate import evaluate

In [16]:
model_dir = 'experiments/base_model'
json_path = os.path.join(model_dir, 'params.json')
params = utils.Params(json_path)

In [17]:
params.dict

{'learning_rate': 0.001,
 'batch_size': 1,
 'num_epochs': 1,
 'dropout_rate': 0.8,
 'num_channels': 32,
 'save_summary_steps': 100,
 'num_workers': 0,
 'cuda': 0}

In [18]:
torch.manual_seed(230);

In [19]:
dataloaders = data_loader.fetch_dataloader(['train', 'val'], params)
train_dl = dataloaders['train']
val_dl = dataloaders['val']

In [20]:
len(train_dl), len(val_dl)

(25, 5)

In [26]:
img, target = next(iter(train_dl))

In [27]:
target.shape

torch.Size([1, 1, 512, 512])

In [28]:
target.squeeze[:5, :5]

TypeError: 'builtin_function_or_method' object is not subscriptable

In [21]:
model = net.Unet()

In [22]:
model = net.Unet()
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
loss_fn = net.loss_fn
metrics = net.metrics

In [ ]:
train(model, optimizer, loss_fn, train_dl, metrics, params)